## Unveiling Pulsars: Machine Learning for Pulsar Identification

#### Introduction

The HTRU2 dataset comes from a project that looks for rare stars called pulsars, which send radio signals to Earth. These stars are essential for understanding space and the relation between stars.

Pulsars spin quickly, and as they do, they send out a repeating radio signal that can be detected using radio telescopes. This signal is like a specific code for each pulsar, but it's hard to find because there's a lot of interference and noise.

The dataset we'll use includes measurements like the average pulse, its variability, shape, and other features. It has a total of 17,898 examples. Among these, 1,639 are positive (real pulsars), and 16,259 are negative (noise). The data comes in two formats, CSV and ARFF, and each entry has a set of values for these measurements, with 0 indicating noise and 1 indicating a real pulsar.

To discover hidden pulsars, we use Python that learns from examples. We tell the computer what real pulsars look like and what noise looks like. Then, it tries to sort new examples into "pulsar" or "not pulsar." Our goal is to create a program that can tell the difference between pulsars and noise, which will help us discover more about these incredible stars.

#### Methods and Results

- *describe in written English the methods you used to perform your analysis from beginning to end that narrates the code the does the analysis.*
- your report should include code which:
    - *loads data from the original source on the web*
    - *wrangles and cleans the data from it's original (downloaded) format to the format necessary for the planned analysis*
    - *performs a summary of the data set that is relevant for exploratory data analysis related to the planned analysis*
    - *creates a visualization of the data set that is relevant for exploratory data analysis related to the planned analysis*
    - *performs the data analysis*
    - *creates a visualization of the analysis*

*note: all tables and figure should have a figure/table number and a legend*

Import packages needed for analysis

In [1]:
# might not need all these - feel free to edit
import random

import altair as alt
import pandas as pd
import numpy as np
from sklearn import set_config
from sklearn.compose import make_column_transformer
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.utils import resample


# Simplify working with large datasets in Altair
alt.data_transformers.disable_max_rows()

# Output dataframes instead of arrays
set_config(transform_output="pandas")

First, we need to load the data set from the web using `read_csv` and name it `pulsar_data`

In [2]:
# Import dataset
pulsar_data = pd.read_csv("https://github.com/Tikio88/Group22_Project/blob/main/data/HTRU_2.csv?raw=true")

Then, we will add column labels so our data set is easier to understand. We will also edit the `pulsar?` column using the `replace` method to replace the values from `'0'` to `'Noise'` and `'1'` to `'Pulsar'`.

In [3]:
# Add column names
pulsar_data.columns = ["mean_integrated_profile", 
                       "standard_deviation_integrated_profile", 
                       "excess_kurtosis_integrated_profile", 
                       "skewness_integrated_profile", 
                       "mean_dm-snr_curve", 
                       "standard_deviation_dm-snr_curve",
                       "excess_kurtosis_dm-snr_curve",
                       "skewness_dm-snr_curve",
                       "label"]

# make "pulsar?" column variable more readable for analysis
pulsar_data["label"] = pulsar_data["label"].replace({0: "Noise", 1:"Pulsar"})

pulsar_data.head(20).style.set_caption("Table 1.0: Tidy Pulsar DataFrame")

,mean_integrated_profile,standard_deviation_integrated_profile,excess_kurtosis_integrated_profile,skewness_integrated_profile,mean_dm-snr_curve,standard_deviation_dm-snr_curve,excess_kurtosis_dm-snr_curve,skewness_dm-snr_curve,label
0,102.507812,58.882430,0.465318,-0.515088,1.677258,14.860146,10.576487,127.393580,Noise
1,103.015625,39.341649,0.323328,1.051164,3.121237,21.744669,7.735822,63.171909,Noise
2,136.750000,57.178449,-0.068415,-0.636238,3.642977,20.959280,6.896499,53.593661,Noise
3,88.726562,40.672225,0.600866,1.123492,1.178930,11.468720,14.269573,252.567306,Noise
4,93.570312,46.698114,0.531905,0.416721,1.636288,14.545074,10.621748,131.394004,Noise
5,119.484375,48.765059,0.031460,-0.112168,0.999164,9.279612,19.206230,479.756567,Noise
6,130.382812,39.844056,-0.158323,0.389540,1.220736,14.378941,13.539456,198.236457,Noise
7,107.250000,52.627078,0.452688,0.170347,2.331940,14.486853,9.001004,107.972506,Noise
8,107.257812,39.496488,0.465882,1.162877,4.079431,24.980418,7.397080,57.784738,Noise
9,142.078125,45.288073,-0.320328,0.283953,5.376254,29.009897,6.076266,37.831393,Noise


Once the data has been imported, it is important to confirm whether the data is tidy, otherwise we could run into errors further down in our analysis.

Visually, we can see that the pulsar_data dataframe fulfills the three pillars of being tidy data. 
1. each row is a single observation
2. each column is a single variable, and 
3. each value is a single cell.

Finally, we will confirm whether there are any empty cells in our dataframe. 

In [4]:
# Confirms that no cells are empty. 

pulsar_data.isnull().sum().sum()

0

#comment

In [5]:
#  Count the amount of pulsar examples and the amount of noise examples in our data. 

pulsar_data_count = pulsar_data["label"].value_counts().to_frame()
pulsar_data_count.columns = ["count"]
pulsar_data_count.style.set_caption("Table 1.1: Label Count")

,count
label,
Noise,16258
Pulsar,1639


In [6]:
# Provide the percentage between the amount of pulsar examples and noise examples in our data. 

pulsar_data_percentage = pulsar_data["label"].value_counts(normalize=True).to_frame()
pulsar_data_percentage.columns = ["percentage"]
pulsar_data_percentage["percentage"] = pulsar_data_percentage["percentage"] * 100
pulsar_data_percentage.style.set_caption("Table 1.2: Label Count by Percentage")

,percentage
label,
Noise,90.842041
Pulsar,9.157959


Now, let us complete an analysis on the variables that we may use in classifying whether we have found a pulsar.

In [7]:
pulsar_data.loc[:, :"skewness_dm-snr_curve"].describe()

,mean_integrated_profile,standard_deviation_integrated_profile,excess_kurtosis_integrated_profile,skewness_integrated_profile,mean_dm-snr_curve,standard_deviation_dm-snr_curve,excess_kurtosis_dm-snr_curve,skewness_dm-snr_curve
count,17897.000000,17897.000000,17897.000000,17897.000000,17897.000000,17897.000000,17897.000000,17897.000000
mean,111.078321,46.549021,0.477897,1.770417,12.614926,26.326918,8.303574,104.859419
std,25.652705,6.843040,1.064056,6.168058,29.473637,19.471042,4.506217,106.517270
min,5.812500,24.772042,-1.876011,-1.791886,0.213211,7.370432,-3.139270,-1.976976
25%,100.929688,42.375426,0.027108,-0.188528,1.923077,14.437330,5.781485,34.957119
50%,115.078125,46.946435,0.223241,0.198736,2.801839,18.459977,8.433872,83.068996
75%,127.085938,51.022887,0.473349,0.928206,5.464883,28.428152,10.702973,139.310905
max,192.617188,98.778911,8.069522,68.101622,223.392141,110.642211,34.539844,1191.000837


As an exploratory visualization of the data, we created a graph comparing the Skewness of the integrated profile (y-axis) to the Excess kurtosis of the integrated profile (x-axis). We also coloured the pulsar and noise examples to be able to determine if we can obtain any information on when we have a pulsar, or when it is noise. Based on the graph, we found that lower values of the Skewness of the integrated profile and of the Excess kurtosis of the integrated profile are typically noise, while pulsars typically have larger values. From the visualization of our graph and tables 1.1 and 1.2, we were able to determine that we have an imbalance between the number of Noise and Pulsar Labels. 

In [8]:
# Create the graph to visualize our data.
pulsar_plot = alt.Chart(pulsar_data.sample(n=2000)).mark_circle(opacity=0.4).encode(
    x=alt.X("excess_kurtosis_integrated_profile").title("Excess kurtosis of the integrated profile"),
    y=alt.Y("skewness_integrated_profile").title("Skewness of the integrated profile"),
    color=alt.Color("label").title("label")
)
pulsar_plot

alt.Chart(...)

<h2>Preprocessing The Data</h2

When reviewing our data we see that the amount of labels classified as Noise is much greater than the amount of confirmed Pulsars. This issue is called <b>class imbalance</b>.  Since classifiers like the 
K-nearest neighbor algorithm use the labels of nearby points to predict the label of a new point, if there are many more data points with one label overall, such as in our pulsar_data, the algorithm is more likely to pick that label in general. Therefor, we rebalanced the data by oversampling the rare label. 

In [9]:
## columns to be used for integrated training
integrated_columns=["mean_integrated_profile", 
                        "standard_deviation_integrated_profile", 
                        "excess_kurtosis_integrated_profile",
                        "skewness_integrated_profile"] 

## columns to be used for snr training
snr_columns=["mean_dm-snr_curve", 
                        "standard_deviation_dm-snr_curve", 
                        "excess_kurtosis_dm-snr_curve",
                        "skewness_dm-snr_curve"]



pulsar_data.columns = ["mean_integrated_profile", 
                       "standard_deviation_integrated_profile", 
                       "excess_kurtosis_integrated_profile", 
                       "skewness_integrated_profile", 
                       "mean_dm-snr_curve", 
                       "standard_deviation_dm-snr_curve",
                       "excess_kurtosis_dm-snr_curve",
                       "skewness_dm-snr_curve",
                       "label"]

# make "pulsar?" column variable more readable for analysis
pulsar_data["label"] = pulsar_data["label"].replace({0: "Noise", 1:"Pulsar"})



noise_label = pulsar_data[pulsar_data["label"] == "Noise"]
pulsar_label = pulsar_data[pulsar_data["label"] == "Pulsar"]
pulsar_label_upsample = resample(
    pulsar_label, n_samples=noise_label.shape[0]
)

upsampled_pulsar_data = pd.concat((pulsar_label_upsample, noise_label))
upsampled_pulsar_data['label'].value_counts().to_frame().style.set_caption("Table 1.3: Label Count After Oversampling")

,count
label,
Pulsar,16258
Noise,16258


Once the class imbalance has been corrected, we finalized preprocessing our data by standardizing the data. We did this for the integrated_profile and the dm_snr_curve data independently to be able to verify if there are any differences if we use one or the other to classify our data.

In [10]:
## integrated preprocessor
polestar_preprocessor_integrated = make_column_transformer(
    (StandardScaler(), integrated_columns),)

## snr preprocessor
polestar_preprocessor_snr = make_column_transformer(
    (StandardScaler(), snr_columns),)



<h2>Optimizing the Algorithm</h2>

Upon completing the preprocessing step for the data, we began the process to utilize the data. Our k nearest neighbour (knn) gets initialized unspecified, to be used when we create the pipelines for the integrated profile and the dm-snr curve data columns. The parameter grid is then initialized to be used in the GridSearchCV function that will be used to identify the best value of our nearest neighbour.We chose the range of our parameter grid to go from 1 to 12, with steps of 1, to be able to evaluate the optimal value of our knn.

In [11]:
## kNeighborsClassifier we will use in the future in gridSearchCV
knn = KNeighborsClassifier()

## making the integrated pipeline with integrated preprocessor and the standard knn
polestar_integrated_pipe = make_pipeline(polestar_preprocessor_integrated, knn)

## making the snr pipeline with snr preprocessor and the standard knn
polestar_snr_pipe = make_pipeline(polestar_preprocessor_snr, knn)


## these are the values of n we will use in the future, starting at 1, going up to 12 in steps of 1.
parameter_grid = {
    "kneighborsclassifier__n_neighbors": range(1, 12, 1),
}

## setting up Gridsearch:
The following code creates two tune grids, one for the Integrated profile and one for the snr profile. Even though it may at first seem like too much to use 10 cuts, considering after the balancing our 16258 points of both noise and pulsar entries, we consider it to be a good balance between performance and accuracy. We then create 2 variables, accuracies_grid_integrated and accuracies_grid_snr which consist of columns from all the test results, and which n parameter we used, as well as mean, standard deviation, and a couple other statistics. For our process we will only use the n neighbors and mean_test_score columns to get an idea of which n gives us the best evaluation performance.score.

In [12]:
## setting up the grid search for integrated, with 10 cuts.
integrated_tune_grid = GridSearchCV(
    estimator=polestar_integrated_pipe,
    param_grid=parameter_grid,
    cv=10
)


## setting up the grid search for snr, with 10 cuts.
snr_tune_grid = GridSearchCV(
    estimator=polestar_snr_pipe,
    param_grid=parameter_grid,
    cv=10
)

## performing the grid search and fit for integrated
accuracies_grid_integrated = pd.DataFrame(
    integrated_tune_grid.fit(
        upsampled_pulsar_data[integrated_columns],
        upsampled_pulsar_data["label"]
    ).cv_results_
)

## performing the grid search and fit for snr
accuracies_grid_snr = pd.DataFrame(
    snr_tune_grid.fit(
        upsampled_pulsar_data[snr_columns],
        upsampled_pulsar_data["label"]
    ).cv_results_
)

## graphing our findings:
I first add another column to accuracies_grid_integrated and accuracies_grid_snr indicating whether we're looking at the integrated mean, or the snr mean. I later use this in the color section of mark_line as the titles for the blue and orange graphs as can be seen below. I used a mark_line since we're trying to see a trend between the different n values which in this case is decreasing starting from n=1.

In [21]:
accuracies_grid_integrated['model'] = 'Integrated mean'
accuracies_grid_snr['model'] = 'SNR mean'

accuracy_vs_k_integrated = alt.Chart(accuracies_grid_integrated, title="n vs mean").mark_line(point=True).encode(
    x=alt.X("param_kneighborsclassifier__n_neighbors").title("Neighbors"),
    y=alt.Y("mean_test_score")
        .scale(domain=(0.9, 1))
        .title("Accuracy estimate"),
    color=alt.Color("model")
)+ alt.Chart(accuracies_grid_snr).mark_line(point=True).encode(
    x=alt.X("param_kneighborsclassifier__n_neighbors").title("Neighbors"),
    y=alt.Y("mean_test_score")
        .scale(domain=(0.9, 1))
        .title("Accuracy estimate"),
    color=alt.Color("model")
).properties(width = 500, height = 500)
accuracy_vs_k_integrated

alt.LayerChart(...)

#### Discussion

- *summarize what you found*
- *discuss whether this is what you expected to find?*
- *discuss what impact could such findings have?*
- *discuss what future questions could this lead to?*

#### References

- *two references used in discussion*
- *reference of data set from web*